In [1]:
import pandas as pd
import requests
import random
from bs4 import BeautifulSoup
from docxtpl import DocxTemplate
import os
from pathlib import Path
import time

C:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\docxcompose\properties.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [8]:
def get_economics_universities():
    """从IDEAS经济学排名网站爬取大学数据，并选择30所大学"""
    print("正在从IDEAS经济学排名网站爬取大学数据...")
    
    try:
        url = "http://ideas.repec.org/top/top.econdept.html"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        universities = []
        
        table = soup.find('table', class_='shorttop')
        if table:
            rows = table.find_all('tr')[1:]  # 跳过表头
            
            for row in rows:
                cells = row.find_all('td')
                if len(cells) >= 2:
                    institution_link = cells[1].find('a')
                    if institution_link:
                        university_name = institution_link.get_text(strip=True)
                        
                        # 清理机构名称
                        if ',' in university_name:
                            parts = university_name.split(',')
                            if len(parts) > 1:
                                university_name = parts[1].strip()
                        
                        # 特殊处理：Alma Mater Studiorum - Università di Bologna
                        if university_name == "Alma Mater Studiorum - Università di Bologna":
                            university_name = "Università di Bologna"

                        if university_name and university_name not in universities:
                            universities.append(university_name)
                            
                        # 收集前90所大学
                        if len(universities) >= 90:
                            break
        
        if len(universities) < 90:
            print(f"警告：只爬取到 {len(universities)} 所大学，不足90所")
            return None
        
        # 从三个区间各选择前10所大学
        front_30 = universities[:30]
        middle_30 = universities[30:60]
        back_30 = universities[60:90]
        
        selected_front = front_30[0:10]
        selected_middle = middle_30[0:10]
        selected_back = back_30[0:10]
        
        selected_universities = selected_front + selected_middle + selected_back
        
        return selected_universities
        
    except Exception as e:
        print(f"爬取大学数据时出错: {e}")
        return None

# something I learn from the process of crawler:
#  try:
#       url = "the website we want to crawl"
#       # 设置请求头模仿浏览器行
#       headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
#       response = requests.get(url, headers=headers, timeout=30)
#       response.raise_for_status()
#       # ... 解析代码
#  except Exception as e:
#       #统一的错误处理和日志记录
#       print(f"爬取大学数据时出错: {e}")
#       return None

In [9]:
def get_research_areas():
    """获取研究领域、期刊、职业和技能数据"""
    research_data = {
        'Research Area': ['Economics', 'Finance', 'Market'],
        'Journals': [
            'American Economic Review(AER), Quarterly Journal of Economics(QJE), Econometrica',
            'Journal of Finance, Journal of Financial Economics, Review of Financial Studies',
            'Marketing Science, Journal of Marketing, Journal of Marketing Research'
        ],
        'Career': ['economic analyst', 'quant developer', 'marketing manager'],
        'Skills': [
            'Microsoft Powerpoint, Mandarin, Power BI, Microsoft Excel, Auditing, etc',
            'Java, C++, Python, computer networking, R, SQL, relational databases, etc',
            'Power BI, writing skills, digital marketing, organizational skills, marketing, etc'
        ]
    }
    
    return research_data

In [10]:
def save_data_to_csv(universities, research_data):
    """保存数据到CSV文件"""
    # 保存大学列表
    df_universities = pd.DataFrame(universities, columns=['University'])
    csv_filename1 = "excel_list1.csv"
    df_universities.to_csv(csv_filename1, index=False, encoding='utf-8-sig')
    print(f"大学列表已保存到: {csv_filename1}")
    
    # 保存研究领域数据
    df_research = pd.DataFrame(research_data)
    csv_filename2 = "excel_list2.csv"
    df_research.to_csv(csv_filename2, index=False, encoding='utf-8-sig')
    print(f"研究领域、期刊、职业和技能已保存到: {csv_filename2}")
# something I learn from the process of saving a csv file:
# 创建DataFrame → 保存为CSV → 指定编码 → 不保存索引

In [5]:
def generate_sop_documents(universities, research_data, personal_info, output_dir_word):
    """生成所有SOP Word文档"""
    print("\n开始生成Word文档...")
    
    # 加载模板
    template_path = r"D:\desktop\sop_template.docx"
    doc = DocxTemplate(template_path)
    print(f"成功加载模板: {template_path}")
    
    generated_files = []
    
    for university in universities:
        for i, research_area in enumerate(research_data['Research Area']):
            # 准备上下文数据
            journals = research_data['Journals'][i].split(', ')
            career_goal = research_data['Career'][i]
            skills = research_data['Skills'][i]
            
            context = {
                'university': university,
                'research_area': research_area,
                'journal1': journals[0] if len(journals) > 0 else '',
                'journal2': journals[1] if len(journals) > 1 else '',
                'journal3': journals[2] if len(journals) > 2 else '',
                'career_goal': career_goal,
                'skills': skills,
                'name': personal_info['name']
            }
            
            # 渲染模板
            doc.render(context)
            
            # 保存Word文档
            filename_base = f"SOP_{university.replace(' ', '_')}_{research_area}"
            word_filename = f"{filename_base}.docx"
            word_filepath = output_dir_word / word_filename
            doc.save(word_filepath)
            
            generated_files.append((word_filepath, filename_base))
            print(f"生成Word文档: {word_filename}")
    
    return generated_files

In [6]:
def convert_to_pdf(generated_files, output_dir_pdf):
    """将Word文档转换为PDF"""
    if os.name != 'nt':  # 非Windows系统
        print("PDF转换仅支持Windows系统")
        return 0
    
    print("\n开始转换为PDF格式...")
    pdf_count = 0
    
    try:
        from docx2pdf import convert
        
        for word_filepath, filename_base in generated_files:
            max_retries = 3
            retry_count = 0
            success = False
            
            while retry_count < max_retries and not success:
                try:
                    pdf_filename = f"{filename_base}.pdf"
                    pdf_filepath = output_dir_pdf / pdf_filename
                    
                    # 添加延迟，避免资源冲突
                    if retry_count > 0:
                        time.sleep(2)  # 重试时等待2秒
                    
                    convert(word_filepath, pdf_filepath)
                    pdf_count += 1
                    print(f"转换为PDF: {pdf_filename}")
                    success = True
                    
                except Exception as pdf_error:
                    retry_count += 1
                    if retry_count < max_retries:
                        print(f"PDF转换失败 {filename_base}，第{retry_count}次重试: {pdf_error}")
                    else:
                        print(f"PDF转换失败 {filename_base}，已重试{max_retries}次: {pdf_error}")
        
        return pdf_count
        
    except ImportError:
        print("无法导入docx2pdf库，请确保已安装")
        return 0
    except Exception as e:
        print(f"PDF转换过程中出错: {e}")
        return 0

In [7]:
def main():
    """主函数：生成90份申请信"""
    # 步骤10: 创建输出目录
    home_dir = Path.home()
    output_dir = home_dir / "HW_School_Application"
    output_dir.mkdir(exist_ok=True)
    
    # 子目录
    word_dir = output_dir / "word_documents"
    pdf_dir = output_dir / "pdf_documents"
    word_dir.mkdir(exist_ok=True)
    pdf_dir.mkdir(exist_ok=True)
    
    # 个人信息
    personal_info = {
        'name': "ZHAO WEI", 
    }
    
    print("=" * 50)
    print("留学申请信自动生成器")
    print("=" * 50)
    
    # 获取数据
    universities = get_economics_universities()   
    research_data = get_research_areas()

    # 保存数据到CSV
    save_data_to_csv(universities, research_data)
    
    # 生成所有Word文档
    generated_files = generate_sop_documents(universities, research_data, personal_info, word_dir)
    
    # 转换为PDF
    pdf_count = convert_to_pdf(generated_files, pdf_dir)
    
    # 结果汇总
    print("\n" + "=" * 50)
    print("生成完成!")
    print("=" * 50)
    print(f"成功生成 {len(generated_files)} 个Word文档")
    print(f"成功转换 {pdf_count} 个PDF文档")
    print(f"Word文档保存在: {word_dir}")
    print(f"PDF文档保存在: {pdf_dir}")
    
    # 显示部分生成的文档作为示例
    print("\n部分生成的文档:")
    for i, (_, filename_base) in enumerate(generated_files[:5]):
        print(f"{i+1}. {filename_base}")

if __name__ == "__main__":
    main()

大学申请信生成器
正在从IDEAS经济学排名网站爬取大学数据...
大学列表已保存到: excel_list1.csv
研究领域、期刊、职业和技能已保存到: excel_list2.csv

开始生成Word文档...
成功加载模板: D:\desktop\sop_template.docx
生成Word文档: SOP_Harvard_University_Economics.docx
生成Word文档: SOP_Harvard_University_Finance.docx
生成Word文档: SOP_Harvard_University_Market.docx
生成Word文档: SOP_Massachusetts_Institute_of_Technology_(MIT)_Economics.docx
生成Word文档: SOP_Massachusetts_Institute_of_Technology_(MIT)_Finance.docx
生成Word文档: SOP_Massachusetts_Institute_of_Technology_(MIT)_Market.docx
生成Word文档: SOP_University_of_California-Berkeley_Economics.docx
生成Word文档: SOP_University_of_California-Berkeley_Finance.docx
生成Word文档: SOP_University_of_California-Berkeley_Market.docx
生成Word文档: SOP_University_of_Chicago_Economics.docx
生成Word文档: SOP_University_of_Chicago_Finance.docx
生成Word文档: SOP_University_of_Chicago_Market.docx
生成Word文档: SOP_Paris_School_of_Economics_Economics.docx
生成Word文档: SOP_Paris_School_of_Economics_Finance.docx
生成Word文档: SOP_Paris_School_of_Economics_Market.docx
生成Word文档

C:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.32s/it]


转换为PDF: SOP_Harvard_University_Economics.pdf


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


转换为PDF: SOP_Harvard_University_Finance.pdf


...

转换为PDF: SOP_Rutgers_University-New_Brunswick_Finance.pdf


  0%|                                                                                            | 0/1 [00:02<?, ?it/s]

PDF转换失败 SOP_Rutgers_University-New_Brunswick_Market，第1次重试: Word.Application.Documents



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.62s/it]

转换为PDF: SOP_Rutgers_University-New_Brunswick_Market.pdf

生成完成!
成功生成 90 个Word文档
成功转换 90 个PDF文档
Word文档保存在: C:\Users\lenovo\HW_School_Application\word_documents
PDF文档保存在: C:\Users\lenovo\HW_School_Application\pdf_documents

部分生成的文档:
1. SOP_Harvard_University_Economics
2. SOP_Harvard_University_Finance
3. SOP_Harvard_University_Market
4. SOP_Massachusetts_Institute_of_Technology_(MIT)_Economics
5. SOP_Massachusetts_Institute_of_Technology_(MIT)_Finance
